## 深層学習特論レポート

氏名: 大北 
学生番号: 20670000  

paper with codeの分類: (Representation Learning)  
論文のURL: (どこどこのURL)  
githubのURL: (github.com/tsuyoshi-okita/class/Skipgram.ipynb)  
deep learning libraryの枠組み: kerasでtensorflow backend   

(jupyterで改行を行なうには, 文末にスペースを2つ入れます)

以下の例はpaper with codeのコードではないので, 上記の分類とgithubのURLでは
ありませんが, みなさんのレポートではここに該当する論文のURLやgithubのURLを
paper with codeから書いてください.


## モデルのサマリ

ここではSkipGramのコードのサマリを以下のようにして表示させました.

ここでポイントはオリジナルコードにはありませんが、vocab_inputを入力させるためにXtrainとYtrainを定義した部分です. これによりsummaryが出力できました。


In [10]:
%matplotlib inline
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer, one_hot
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import nltk
import numpy as np
#import operator
import codecs

Xtrain= np.array([[1.,0.],[1.,0.]])
Ytrain = np.array([[1.,0.,0.],[0.,0.,1.]])
                   
vocab_input=Xtrain.shape[1]
vocab_output=Ytrain.shape[1]

model = Sequential()
model.add(Dense(300, input_shape=(vocab_input,)))
#model.add(Dense(10, input_shape=(vocab_input,)))
model.add(Activation("relu"))
model.add(Dropout(0.3))
model.add(Dense(vocab_output))
model.add(Activation("softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 300)               900       
_________________________________________________________________
activation_1 (Activation)    (None, 300)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 903       
_________________________________________________________________
activation_2 (Activation)    (None, 3)                 0         
Total params: 1,803
Trainable params: 1,803
Non-trainable params: 0
_________________________________________________________________


## 簡単な入力データの例

以下のような形で, 'this is an example sentence.'という文を用いた簡単な例として,   
Xtrain=[[1. 0.]  
 [1. 0.]]  
Ytrain=[[1. 0. 0.]  
 [0. 0. 1.]]  
という設定を行なった.

In [4]:
!cat sampleText.txt

this is an example sentence.


In [16]:
infile = codecs.open("./sampleText.txt", "r", 'utf-8')
text = infile.read()
sents = nltk.sent_tokenize(text)
tokenizer = Tokenizer(5000)  
tokens = tokenizer.fit_on_texts(sents)
vocab_size = len(tokenizer.word_counts) + 1
#print ('vocab_size=', vocab_size)
xs = []
ys = []
for sent in sents:
    embedding = one_hot(sent, vocab_size)
    triples = list(nltk.trigrams(embedding))
    for x in triples:
        left_word = x[0]
        center_word = x[1]
        right_word = x[2]
        xs.append(center_word)
        ys.append(left_word)
        xs.append(center_word)
        ys.append(right_word)

xs=np.array(xs).reshape(-1,1)
ys=np.array(ys).reshape(-1,1)
ohe = OneHotEncoder()
ohe2 = OneHotEncoder()
X = ohe.fit_transform(xs).todense()
#print (ohe.fit_transform(np.array(1).reshape(-1,1)))
Y = ohe2.fit_transform(ys).todense()
#print (X)
Xtrain0, Xtest, Ytrain0, Ytest = train_test_split(X, Y, test_size=0.3, random_state=42)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain0, Ytrain0, test_size=0.3, random_state=42)
#print (Xtrain.shape)
print ('Xtrain=', Xtrain)
print ('Ytrain=', Ytrain)

Xtrain= [[1. 0.]
 [1. 0.]]
Ytrain= [[1. 0. 0.]
 [0. 0. 1.]]


## 簡単なデータでモデルを動作させる例

トレーニングを行うfitを動作させると以下のようになる.


In [15]:
BATCH_SIZE = 2
NUM_EPOCHS = 2
history = model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS, verbose=1,
                   validation_data = [Xval, Yval])

Train on 2 samples, validate on 2 samples
Epoch 1/2
2/2 [==============================] - 0s 4ms/step - loss: 1.0530 - accuracy: 0.5000 - val_loss: 1.0712 - val_accuracy: 0.5000
Epoch 2/2
2/2 [==============================] - 0s 2ms/step - loss: 0.9423 - accuracy: 1.0000 - val_loss: 1.0737 - val_accuracy: 0.5000
